Essentially, I am determining session length from an nginx log file.

[Some of this explanation may not be completely correct, but the technical parts relating to our problem are correct.]

WMRA streams its content over the Internet. They have to report on the number and length of sessions.

They have two streaming servers; icecast and nginx. Some folks on the Internet have created a script that can generate the necessary report data from the icecast log files.

There is an expensive add-on to the nginx server that generates the data, but is out of their budget.

I offered to help them take the information from the nginx server logs and generate something that looks like the icecast logs.

Then, WMRA can just use the Internet script to generate the actual reports.

To listen to a stream, the client software makes a request for an index file which has information about the next chunk of stream. This happens every 40 seconds.

So, for each client, we just need to look for breaks in the 40 second requests. Then we can determine the length of their session.

In [ ]:
import pandas as pd
import re

In [ ]:
from datetime import timedelta

In [ ]:
# sample data
#
# the first line is a request for an index file
# the second is the next chunk of stream
#
# 96.249.243.92 - - [03/Oct/2016:03:20:11 -0400] "GET /hls/wmra_hi/index.m3u8 HTTP/1.1" 200 128 "-" "AppleCoreMedia/1.0.0.14A456 (iPhone; U; CPU OS 10_0_2 like Mac OS X; en_us)"
# 96.249.243.92 - - [03/Oct/2016:03:20:12 -0400] "GET /hls/wmra_hi/32291.ts HTTP/1.1" 200 1161464 "-" "AppleCoreMedia/1.0.0.14A456 (iPhone; U; CPU OS 10_0_2 like Mac OS X; en_us)"

# from http://nginx.org/en/docs/http/ngx_http_log_module.html#log_format
# log_format combined '$remote_addr - $remote_user [$time_local] '
#                     '"$request" $status $body_bytes_sent '
#                     '"$http_referer" "$http_user_agent"';

In [ ]:
# here is a regex that matches a valid GET request 
# and grabs each of the fields from the request
# ip, user, timestamp, request, status, size, referer, agent
regex = re.compile('(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}) - (.+) \[(.+)\] "GET (.+)" (\d{3}) (\d+) "(.*)" "(.+)"')

In [ ]:
# create a dataframe from the nginx log file
def get_df_from_log(log):
    rows = []
    columns = ['ip', 'user', 'timestamp', 'request', 'status', 'size', 'referer', 'agent']
    misses = []
    with open(log) as myfile:
        for line in myfile:
            # df only contains valid GET requests
            # misses contains invalid or non GET requests
            match = regex.match(line)
            if match:
                row = {'ip': match.group(1), 
                       'user': match.group(2),
                       'timestamp': match.group(3),
                       'request': match.group(4), 
                       'status': match.group(5), 
                       'size': match.group(6), 
                       'referer': match.group(7), 
                       'agent': match.group(8)}
                rows.append(row)
            else:
                misses.append(line)
    df = (pd.DataFrame(rows, columns=columns))
    return (df, misses)

In [ ]:
df, misses = get_df_from_log('data/nginxlog')

In [ ]:
# there are 47992 rows and 8 columns
df.shape

In [ ]:
df.head()

In [ ]:
# len(misses)

In [ ]:
# misses

In [ ]:
# reqs = df['request'].unique()

In [ ]:
# len(reqs)

In [ ]:
# reqs

In [ ]:
# we are only interested in the index requests
df = df[df['request'].str.contains('index')]

In [ ]:
# we are down to 19669 rows
df.shape

In [ ]:
# create a datetime column from the timestamp

def make_datetime(df):
    df = df.copy()
    # fix timestamp string for pd.to_datetime
    # change colon between date and time to space
    df['timestamp'] = df['timestamp'].str.replace(':', ' ', 1)
    
    # add datetime column from timestamp
    # this takes a little while
    df['datetime'] = pd.to_datetime(df['timestamp'])
    
    return df

In [ ]:
df = make_datetime(df)

In [ ]:
# we added a column
df.shape

In [ ]:
df.head()

In [ ]:
# sort the dataframe by ip address and time of request
df.sort_values(['ip', 'datetime'], inplace=True)

In [ ]:
df.head()

In [ ]:
# add session and duration columns with default values
df['session'] = 0
df['duration'] = timedelta(0)

In [ ]:
# 2 more columns
df.shape

In [ ]:
df.head()

In [ ]:
# the requests should come every 40 seconds. This allows for some variance
def in_range(delta):
    if delta >= timedelta(seconds=35) and delta <= timedelta(seconds=45):
        return True
    return False

In [ ]:
# add delta and in_range columns
# delta is length of time since last request
df['delta'] = df['datetime'].diff().fillna(0)
# is delta close to 40 seconds?
df['in_range'] = df['delta'].apply(in_range)

In [ ]:
df.shape

In [ ]:
df.head()

Here is my brute force way of solving the problem. It loops through each line. I believe there may be a better way to do this using Pandas grouping features.

In [ ]:
def calculate_session_duration(prepped_df):
    # calculate a timedelta for each session
    # while requests from an ip are in_range, we keep cumsum
    # we increment session when not in_range, and
    # reset session for each ip 
    # add session and duration to the df
    df_copy = prepped_df.copy()
    iterator = df_copy.iterrows()
    _, last = next(iterator)
    ip = last['ip']
    session = 0
    cumsum = timedelta(0)
    for i, row in iterator:
        # reset session count for each client
        if row['ip'] != last['ip']:
            session = 0
        # if same session, calculate cumsum 
        if row['in_range']:
            cumsum += row['delta']
            # print(cumsum)
        # else, reset cumsum and increment session
        else:
            cumsum = timedelta(0)
            session += 1
        # write cumsum and session to df
        df_copy.loc[i, 'duration'] = cumsum
        df_copy.loc[i, 'session'] = session
        # save index and row for comparison
        # _, last = i, row
        last = row
    return df_copy

In [ ]:
# df.loc[19927, 'in_range'] = False

In [ ]:
# this takes a little while
with_duration = calculate_session_duration(df)

In [ ]:
# we haven't modified the shape of the dataframe, just some of the values
with_duration.shape

In [ ]:
# with_duration.loc[with_duration['ip'] == '96.249.243.92', :].head(25)
with_duration.head(50)

In [ ]:
with_duration.tail()

In [ ]:
# df.isnull().sum()

In [ ]:
# df.loc[df['request'].str.contains('index'), 'request'].unique()

In [ ]:
# df.loc[df['request'].str.contains('wxjm'), 'request'].unique()

In [ ]:
# df.loc[df['request'].str.contains('vv'), 'request'].unique()

In [ ]:
# df.loc[df['request'].str.contains('wmra'), 'request'].unique()

In [ ]:
# df.loc[df['request'].str.contains('index'), 'request'].unique()

In [ ]:
# with_duration['session'].unique()